In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from model import Discriminator, Generator
from math import log2
from tqdm import tqdm
import cv2
import numpy as np
import os
from scipy.stats import truncnorm

torch.backends.cudnn.benchmarks = True

In [9]:
starting_img_size = 4
dataset = "celeb_hq_dataset"
checkpoint_generator = "generator.pth"
checkpoint_discriminator = "discriminator.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 1e-3
batch_size = [4, 4, 4, 4, 4, 4, 4, 2, 1]
img_size = 512
img_channels = 3
z_dim = 256
in_channels = 256
l_gp = 10
num_steps = int(log2(img_size /4)) + 1
progressive_epochs = [10] * len(batch_size)
noise = torch.randn(8, z_dim, 1, 1).to(device)
workers = 4

Extra Functions

In [10]:
def gradient_penalty(discriminator, real, fake, alpha, train_step, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    beta = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = real * beta + fake.detach() * (1 - beta)
    interpolated_images.requires_grad_(True)
    mixed_scores = discriminator(interpolated_images, alpha, train_step)
    gradient = torch.autograd.grad(inputs=interpolated_images, outputs=mixed_scores, grad_outputs=torch.ones_like(mixed_scores), create_graph=True, retain_graph=True,)[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {"state_dict": model.state_dict(),"optimizer": optimizer.state_dict(),}
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location="cuda")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

def plot_to_tensorboard(writer, disc_loss, loss_gen, real, fake, tensorboard_step):
    writer.add_scalar("Loss discriminator", disc_loss, global_step=tensorboard_step)

    with torch.no_grad():
        img_grid_real = torchvision.utils.make_grid(real[:8], normalize=True)
        img_grid_fake = torchvision.utils.make_grid(fake[:8], normalize=True)
        writer.add_image("Real", img_grid_real, global_step=tensorboard_step)
        writer.add_image("Fake", img_grid_fake, global_step=tensorboard_step)

Creating a loader

In [11]:
def create_loader(image_size):
    transform = transforms.Compose([transforms.Resize((image_size, image_size)), transforms.ToTensor(), transforms.RandomHorizontalFlip(p=0.5), transforms.Normalize( [0.5 for _ in range(img_channels)], [0.5 for _ in range(img_channels)], ), ] )
    batch_sizes = batch_size[int(log2(image_size / 4))]
    data = datasets.ImageFolder(root=dataset, transform=transform)
    loader = DataLoader(data, batch_size=batch_sizes, shuffle=True, num_workers=workers, pin_memory=True,)
    return loader, data

Training

In [12]:
def train(discriminator, gen, loader, dataset, step, alpha, discriminator_optimizer, opt_gen, tensorboard_step, writer, generator_Scaler, discriminator_Scaler,):
    loop = tqdm(loader, leave=True)
    for batch_idx, (real, _) in enumerate(loop):
        real = real.to(device)
        cur_batch_size = real.shape[0]

        noise = torch.randn(cur_batch_size, z_dim, 1, 1).to(device)

        with torch.cuda.amp.autocast():
            fake = gen(noise, alpha, step)
            discriminator_real = discriminator(real, alpha, step)
            discriminator_fake = discriminator(fake.detach(), alpha, step)
            gp = gradient_penalty(discriminator, real, fake, alpha, step, device=device)
            disc_loss = (-(torch.mean(discriminator_real) - torch.mean(discriminator_fake)) + l_gp * gp + (0.001 * torch.mean(discriminator_real ** 2)) )

        discriminator_optimizer.zero_grad()
        discriminator_Scaler.scale(disc_loss).backward()
        discriminator_Scaler.step(discriminator_optimizer)
        discriminator_Scaler.update()

        with torch.cuda.amp.autocast():
            gen_fake = discriminator(fake, alpha, step)
            loss_gen = -torch.mean(gen_fake)

        opt_gen.zero_grad()
        generator_Scaler.scale(loss_gen).backward()
        generator_Scaler.step(opt_gen)
        generator_Scaler.update()

        alpha += cur_batch_size / (
            (progressive_epochs[step] * 0.5) * len(dataset)
        )
        alpha = min(alpha, 1)

        if batch_idx % 500 == 0:
            with torch.no_grad():
                fixed_fakes = gen(noise, alpha, step) * 0.5 + 0.5
            plot_to_tensorboard(
                writer,
                disc_loss.item(),
                loss_gen.item(),
                real.detach(),
                fixed_fakes.detach(),
                tensorboard_step,
            )
            tensorboard_step += 1

        loop.set_postfix(
            gp=gp.item(),
            disc_loss=disc_loss.item(),
        )

    return tensorboard_step, alpha

In [13]:
def main():
    gen = Generator(z_dim, in_channels, img_channels=img_channels).to(device)
    discriminator = Discriminator(z_dim, in_channels, img_channels=img_channels).to(device)
    generator_optimizer = optim.Adam(gen.parameters(), lr=lr, betas=(0.0, 0.99))
    discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.0, 0.99))
    discriminator_scaler = torch.cuda.amp.GradScaler()
    generator_scaler = torch.cuda.amp.GradScaler()
    writer = SummaryWriter(f"logs/true_GAN")
    gen.train()
    discriminator.train()

    tensorboard_step = 0
    step = int(log2(starting_img_size / 4))
    for num_epochs in progressive_epochs[step:]:
        alpha = 0
        loader, dataset = create_loader(4 * 2 ** step)
        print(f"Current image size: {4 * 2 ** step}")
        print("----------------------------------------------------------------------------------")
        for epoch in range(num_epochs):
            print(f"Epoch [{epoch+1}/{num_epochs}]")
            tensorboard_step, alpha = train(discriminator,gen, loader, dataset, step, alpha, discriminator_optimizer, generator_optimizer, tensorboard_step, writer, generator_scaler, discriminator_scaler,)
            save_checkpoint(gen, generator_optimizer, filename=checkpoint_generator)
            save_checkpoint(discriminator, discriminator_optimizer, filename=checkpoint_discriminator)

        step += 1  

In [14]:
main()

Current image size: 4
----------------------------------------------------------------------------------
Epoch [1/10]


100%|██████████| 4486/4486 [01:39<00:00, 45.04it/s, disc_loss=0.251, gp=0.00156]   


=> Saving checkpoint
=> Saving checkpoint
Epoch [2/10]


100%|██████████| 4486/4486 [01:37<00:00, 45.97it/s, disc_loss=0.422, gp=0.00296]   


=> Saving checkpoint
=> Saving checkpoint
Epoch [3/10]


...

=> Saving checkpoint
=> Saving checkpoint
Epoch [2/10]


100%|██████████| 4486/4486 [08:02<00:00,  9.30it/s, disc_loss=-2.3, gp=0.00285]    


=> Saving checkpoint
=> Saving checkpoint
Epoch [3/10]


 66%|██████▌   | 2942/4486 [05:26<02:51,  9.01it/s, disc_loss=-.545, gp=0.00362]  


MemoryError: Caught MemoryError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\folder.py", line 230, in __getitem__
    sample = self.loader(path)
  File "c:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\folder.py", line 269, in default_loader
    return pil_loader(path)
  File "c:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\folder.py", line 249, in pil_loader
    return img.c